## Simple example -- pairwise distance

Suppose we have `N` vectors in $\mathbb{R}^3$, stored as a $3 \times N$ matrix and labeled $x_1 \in \mathbb{R}^3$, $x_2 \in \mathbb{R}^3$, ..., $x_N \in \mathbb{R}^3$

Suppose we also have a metric function $\rho: \mathbb{R}^3 \times \mathbb{R}^3 \rightarrow \mathbb{R}^+$  that we would like to apply to each pair of vectors to compute an $N \times N$ matrix $D$, where the $i,j$ element is give by $D_{i,j} = \rho(x_i, x_j)$

Let's start with the Euclidean (L2) norm $\rho(x, y) = \sqrt{\sum_{i=1}^3(x_i - y_i)^2}$

Here's how we might compute this in Julia

In [19]:
N = 1500
X = randn(3, N)  # create the matrix of random numbers drawn from N(0, 1)

3×1500 Array{Float64,2}:
 -0.391776   0.233421   1.06281    …   0.98919    0.877327  -0.766027 
 -1.74568   -0.496354   0.0830358     -0.919035  -0.918698   0.0799679
  0.215267  -0.174774  -1.24064        0.996978  -0.367617  -0.808813 

In [21]:
function computeD(ρ, X)
    m, N = size(X)
    out = zeros(N, N)
    for j in 1:N, i in 1:N
        # [index...] notation. Colon means "everything" in dimension
        out[i, j] = ρ(X[:, i], X[:, j])
    end
    out # implicit return of last expression in function
end

# short-hand function
# the `.+` and `.^` apply "elementwise" to arguments
# Can work with other functions `f` like `f.(arg1, arg2, arg3, ...)`
ρ_l2(x, y) = sqrt(sum((x.-y).^2))

@time D = computeD(ρ_l2, X)

  1.498281 seconds (9.71 M allocations: 840.869 MiB, 3.92% gc time)


1500×1500 Array{Float64,2}:
 0.0       1.45046   2.75312   2.5337   …  1.78927  1.62304   2.12645 
 1.45046   0.0       1.46957   2.36042     1.457    0.793838  1.31645 
 2.75312   1.46957   0.0       2.7381      2.45286  1.34166   1.87913 
 2.5337    2.36042   2.7381    0.0         1.4748   1.72983   3.658   
 2.81576   1.60544   1.78386   2.3082      1.5051   1.67721   2.50934 
 0.904011  1.56532   3.02308   2.64512  …  1.47674  1.93099   2.28774 
 1.78567   0.3992    1.29438   2.24088     1.35603  0.780843  1.53002 
 2.40102   1.12455   1.42747   2.17739     1.347    1.22169   2.09679 
 2.01506   0.746385  0.859161  2.12472     1.63157  0.574048  1.7231  
 2.02436   1.02914   0.824419  2.31626     2.05878  0.72197   1.72467 
 1.61772   0.181647  1.38744   2.37251  …  1.44729  0.83398   1.33852 
 2.40135   1.10658   1.62365   2.61953     1.55257  1.5102    1.81974 
 3.69417   2.5345    2.32126   2.50828     2.16384  2.40618   3.43299 
 ⋮                                      ⋱        

Let's test our work  by showing the diagonal is all zeros:


In [22]:
using LinearAlgebra: diag

# notice  the `.` notation for `abs`
maximum(abs.(diag(D) .- zeros(N)))

0.0

## Python implementation

So that we can gauge performance

In [23]:
using PyCall

In [24]:
py"""
import numpy as np

def ρ_l2(x, y):
    return np.sqrt(np.sum((x - y) ** 2))


def computeD(X, ρ=ρ_l2):
    m, N = X.shape
    out = np.zeros((N, N))
    for i in range(N):
        for j in range(N):
            out[i, j] = ρ(X[:, i], X[:, j])
    return out
"""

py_computeD = py"computeD"

PyObject <function computeD at 0x137813268>

In [25]:
@time py_D = py_computeD(X)

 18.766631 seconds (45 allocations: 17.168 MiB, 0.03% gc time)


1500×1500 Array{Float64,2}:
 0.0       1.45046   2.75312   2.5337   …  1.78927  1.62304   2.12645 
 1.45046   0.0       1.46957   2.36042     1.457    0.793838  1.31645 
 2.75312   1.46957   0.0       2.7381      2.45286  1.34166   1.87913 
 2.5337    2.36042   2.7381    0.0         1.4748   1.72983   3.658   
 2.81576   1.60544   1.78386   2.3082      1.5051   1.67721   2.50934 
 0.904011  1.56532   3.02308   2.64512  …  1.47674  1.93099   2.28774 
 1.78567   0.3992    1.29438   2.24088     1.35603  0.780843  1.53002 
 2.40102   1.12455   1.42747   2.17739     1.347    1.22169   2.09679 
 2.01506   0.746385  0.859161  2.12472     1.63157  0.574048  1.7231  
 2.02436   1.02914   0.824419  2.31626     2.05878  0.72197   1.72467 
 1.61772   0.181647  1.38744   2.37251  …  1.44729  0.83398   1.33852 
 2.40135   1.10658   1.62365   2.61953     1.55257  1.5102    1.81974 
 3.69417   2.5345    2.32126   2.50828     2.16384  2.40618   3.43299 
 ⋮                                      ⋱        

In [26]:
maximum(abs.(D .- py_D))

0.0

On one run on my machine the Julia version took 1.49 seconds whereas the Python one took 18.76 seconds. 

This was a pretty much "free" speedup as the code was almost the same for both languages. It is not uncommon to se factors of 100 or 1000 when moving hand-written Python code to Julia

Note that the code in each example was not meant to reach optimal performance, both implementations could be improved

## Faster Julia version



In [34]:
function computeD_l2(X)
    m, N = size(X)
    out = zeros(N, N)
    @inbounds for j in 1:N, i in 1:N
        # [index...] notation. Colon means "everything" in dimension
        val = 0.0
        @simd for d in 1:m
            val += (X[d, i] - X[d, j])^2
        end
        out[i, j] = sqrt(val)
    end
    out # implicit return of last expression in function
end

computeD_l2 (generic function with 1 method)

In [70]:
@time D2 = computeD_l2(X);

  0.020914 seconds (6 allocations: 17.166 MiB)


In [38]:
maximum(abs.(D .- D2))

8.881784197001252e-16

On my machine this version of code ran the computation in 0.0095 seconds -- that's a 157x speedup over the original Julia and a 1978x speedup over the Python

With Julia it is possible to write "lower level" code and get drastically better performance

Sometimes you might hear that "in Julia you can get C-like performance if you write C-like code"

This is typically not possible in other common data analytics languages like Python, Matlab, or R